In [7]:
import pandas as pd
from epiweeks import Week
from bulletin import __file__ as __root__
from bulletin.data.casos_confirmados import CasosConfirmados

In [8]:
casos_confirmados = CasosConfirmados(force=True, hard=True)
casos_confirmados.shape()

saved checksum: 56625b0d9cabc5be8bb69f8b506ab1db62f14609e617c4b9e132b2eced042432
current checksum: 56625b0d9cabc5be8bb69f8b506ab1db62f14609e617c4b9e132b2eced042432
Tudo certo, nenhuma alteração detectada
Utilizando forcadamente com método update
Atualizando o arquivo {'casos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\casos.pkl', 'obitos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\obitos.pkl'} com o C:\Users\est.joaosilva\covid19datascience\bulletin\tmp\Casos confirmados.xlsx...
Casos excluidos: 0
Obitos excluidos: 0
{'casos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\casos.pkl', 'obitos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\obitos.pkl'} salvo e C:\Users\est.joaosilva\covid19datascience\bulletin\tmp\casos_confirmados_checksum atualizado
{'casos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\casos.pkl', 'obitos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\obitos.pkl'

(929097, 21564)

In [191]:
casos = casos_confirmados.get_casos()

In [192]:
sivep = pd.read_excel('output/TABELA_testes_sivep.xlsx')
notifica = pd.read_excel('output/TABELA_testes_notifica.xlsx')
notifica = notifica.set_index('SE')
sivep = sivep.set_index('SE')

In [193]:
casos['dt_diag'] = casos['dt_diag'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
casos['is'] = casos['is'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

In [194]:
casos['se_diag'] = casos['dt_diag'].apply(lambda x: Week.fromdate(x))
casos.loc[~casos['is'].isnull(), 'se_is'] = casos['is'].loc[~casos['is'].isnull()].apply(lambda x: Week.fromdate(x))

In [195]:
tabela_diag = casos.groupby(['se_diag'])[['nome']].count()

In [196]:
tabela_is = casos.groupby(['se_is'])[['nome']].count()
for row in tabela_is.index.values:
    if row.year < 2020:
        tabela_is = tabela_is.drop(row)

In [197]:
tabela = pd.DataFrame(sivep['Positividade (%)']).rename(columns={'Positividade (%)': 'Positividade SRAG'})
notifica = pd.DataFrame(notifica['Positividade (%)']).rename(columns={'Positividade (%)': 'Positividade SG'})
tabela = tabela.merge(how = 'left', on = tabela.index, right = notifica)
tabela = tabela.set_index('key_0')

In [198]:
tabela_diag = tabela_diag[['nome']].rename(columns={'nome': 'Diagnóstico'})
tabela_is = tabela_is[['nome']].rename(columns={'nome':'Primeiros sintomas'})
index = pd.Series(dtype = 'int64')
for line in tabela_diag.index.values:
    index = index.append(pd.Series(line.cdcformat(), dtype = 'int64'))
# tabela_diag = tabela_diag.set_index(index)
tabela.index.name = None
tabela = tabela.merge(how = 'left', on = tabela.index, right = tabela_diag)
tabela = tabela[['Positividade SRAG', 'Positividade SG', 'Diagnóstico']]
index = pd.Series(dtype = 'int64')
for line in tabela_is.index.values:
    index = index.append(pd.Series(line.cdcformat(), dtype = 'int64'))
tabela_is = tabela_is.set_index(index)
tabela = tabela.merge(how = 'left', on = tabela.index, right = tabela_is)
tabela = tabela[['Positividade SRAG', 'Positividade SG', 'Diagnóstico', 'Primeiros sintomas']]

tabela = tabela.merge(how = 'left', on = tabela.index, right = sivep['Semana Epidemiológica'])
tabela = tabela[['Semana Epidemiológica']].rename(columns={'Semana Epidemiológica': 'SE'})
tabela = tabela[['Primeiros sintomas', 'Diagnóstico', 'Positividade SG', 'Positividade SRAG', 'SE']]
tabela = tabela.set_index('SE')

C:\Users\est.joaosilva\Anaconda3\lib\site-packages\numpy\core\numeric.py:2378: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


KeyError: Int64Index([202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008,
            202009, 202010, 202011, 202012, 202013, 202014, 202015, 202016,
            202017, 202018, 202019, 202020, 202021, 202022, 202023, 202024,
            202025, 202026, 202027, 202028, 202029, 202030, 202031, 202032,
            202033, 202034, 202035, 202036, 202037, 202038, 202039, 202040,
            202041, 202042, 202043, 202044, 202045, 202046, 202047, 202048,
            202049, 202050, 202051, 202052, 202053, 202101, 202102, 202103,
            202104, 202105, 202106, 202107, 202108, 202109, 202110, 202111,
            202112, 202113, 202114, 202115, 202116, 202117],
           dtype='int64')

In [11]:
# total_diag = tabela_diag.sum(0)
# tabela_diag = tabela_diag.append(total_diag, ignore_index = True)

In [12]:
# total_is = tabela_is.sum(0)
# tabela_is = tabela_is.append(total_is, ignore_index = True)

In [13]:
# tabela_casos['rs'] = list(range(1, 23)) + ['Totais']
# tabela_casos = tabela_casos.set_index('rs')

In [14]:
# tabela_obitos['rs'] = list(range(1, 23)) + ['Totais']
# tabela_obitos = tabela_obitos.set_index('rs')

In [52]:
tabela_diag.to_excel('output/tabela_diag.xlsx', index = False)
tabela_is.to_excel('output/tabela_is.xlsx', index = False)